# QuTiP overview - Optimal Control

Alexander Pitchford (agp1@aber.ac.uk)

# Introduction
In quantum control we look to prepare some specific state, effect some state-to-state transfer, or effect some transformation (or gate) on a quantum system. For a given quantum system there will always be factors that effect the dynamics that are outside of our control. As examples, the interactions between elements of the system or a magnetic field required to trap the system. However, there may be methods of affecting the dynamics in a controlled way, such as the time varying amplitude of the electric component of an interacting laser field. And so this leads to the questions; given a specific quantum system with known time-independent dynamics generator (referred to as the *drift* dynamics generators) and set of externally controllable fields for which the interaction can be described by *control* dynamics generators:
1. what states or transformations can we achieve (if any)
2. what is the shape of the control pulse amplitude required to achieve this

These questions are addressed as *controllability* and *quantum optimal control* [1]. The answer to question of *controllability* is determined by the commutability of the dynamics generators and is formalised as the *Lie Algebra Rank Criterion* as discussed in detail in [1]. The solutions to the second question can be determined through optimal control algorithms, or control pulse optimisation. 
![qc_shematic](./images/quant_optim_ctrl.png "Schematic showing the principle of quantum control")

Quantum Control has many applications including NMR, *quantum metrology*, *control of chemical reactions*, and *quantum information processing*.

To explain the physics behind these algorithms we will first consider only finite-dimensional, closed quantum systems.

# Closed Quantum Systems
In closed quantum systems the states can be represented by kets, and the transformations on these states are unitary operators. The dynamics generators are Hamitonians. The combined Hamiltonian for the system is given by

$ H(t) = H_0 + \sum_{j=1} u_j(t) H_j $

where $H_0$ is the drift Hamiltonian and the $H_j$ are the control Hamiltonians. The $u_j$ are time varying amplitude functions of the specific control.

The dynamics of the system are governed by *Schrödingers equation*.

$\newcommand{\ket}[1]{\left|{#1}\right\rangle} \tfrac{d}{dt}\ket{\psi} = -i H(t)\ket{\psi} $

Note we use units where $\hbar=1$ throughout. The solutions to Schrödingers equation are of the form:

$\ket{\psi(t)} = U(t)\ket{\psi_0}$

where $\psi_0$ is the state of the system at $t=0$ and $U(t)$ is a unitary operator on the Hilbert space containing the states. $U(t)$ is a solution to the *Schrödinger operator equation*

$\tfrac{d}{dt}U = -i H(t)U ,\quad U(0) = \mathbb{1}$

We can use optimal control algorithms to determine a set of $u_j$ that will drive our system from $\ket{\psi_0}$ to $\ket{\psi_1}$, this is state-to-state transfer, or drive the system from some arbitary state to a given state $\ket{\psi_1}$, which is state preparation, or effect some unitary transformation $U_{target}$, called gate synthesis. The latter of these is most important in quantum computation.

# The GRAPE algorithm
The GRadient Ascent Pulse Engineering was first proposed in [2]. The implementation in QuTiP pulseoptim is based on a MATLAB implementation called DYNAMO that is described in [3]. Solutions to Schrödingers equation for a time-dependent Hamitonian are not generally possible to obtain analytically. Therefore, a piecewise constant approximation to the pulse amplitudes is made. Time allowed for the system to evolve $T$ is split into $M$ timeslots (typically these are of equal duration), during which the control amplitude is assumed to remain constant. The combined Hamiltonian can then be approximated as:

$H(t) \approx H_{jk} = H_0 + \sum_{j=1}^N u_{jk} H_j\quad$

where $k$ is a timeslot index and $N$ is the number of controls.  The time evolution operator, or propagator, within the timeslot can then be calculated as:

$X_k:=e^{-iH_{jk}\Delta t_k}$

where $\Delta t_k$ is the duration of the timeslot. The full evolution up to any timeslot $k$ (including the full evolution $t_k=T$) can the be calculated

$X(t_k):=X_k X_{k-1}\cdots X_1 X_0$

If the objective is state-to-state transfer then $X_0=\ket{\psi_0}$ and the target $X_{targ}=\ket{\psi_1}$, for gate synthesis $X_0 = U(0) = \mathbb{1}$ and the target $X_{targ}=U_{targ}$.

A *figure of merit* or *fidelity* is some measure of how close the evolution is to the target, based on the  control amplitudes in the timeslots. The typical figure of merit for unitary systems is the normalised overlap of the evolution and the target.

$\newcommand{\tr}[0]{\operatorname{tr}} f_{PSU} = \tfrac{1}{d} \big| \tr \{X_{targ}^{\dagger} X(T)\} \big|$

where $d$ is the system dimension. In this figure of merit the absolute value is taken to ignore any differences in global phase, and $0 < f < 1$. Typically the fidelity error (or infidelty) is more useful, in this case defined as $\varepsilon = 1 - f$.  There are many other objectives and hence figures of merit.

As there are now $N \times M$ variables (the $u_{jk}$) and one parameter to minimise $\varepsilon$, then the problem becomes a finite multi-varible optimisation problem, for which there are many established methods, often refered to as 'hill-climbing' methods. The simplest of these to understand is that of steepest ascent (or descent). The gradient of the fidelity with respect to all the variables is calculated (or approximated) and a step is made in the variable space in the direction of steepest ascent. These methods are called first order gradient methods. This can be easily visualised by considering a method of finding the top of a hill in thick fog, just head in the direction where the ground rises fastest. This analogy also clearly illustrates one of the main challenges in multi-variable optimisation, which is that all methods have a tendancy to get stuck in local maxima. It is hard to determine whether one has found a global maximum or not. In quantum optimal control however we typically know the maximum, and hence typically we look to minimise the infidelity, which has a lower limit of 0. From here on we will only consider optimising for infidelity minima. In the hill walking analogy the step size is roughly fixed to a stride, however in computations this can be chosen. Clearly there is a trade of here between the number of steps (or iterations) required to reach the minima and possibility that we might step over a minima. In practice it is difficult to determine an efficient and effective step size.

The second order differentials of the infidelity with respect to the variables can be used to approximate the local landscape to a parabola. This way a step (or jump) can be made to where the minima would be if it were parabolic. This typically vastly reduces the number of iterations, and removes to need to guess a step size. The method where all the second differentials are calculated is called the *Newton-Raphson* method. However, calculating the second-order differentials (the Hessian matrix) can be computationally expensive, and so there are a class of methods known as *quasi-Newton* that approximate the Hessian based on successive iterations. The most popular of these (in quantum optimal control) is the Broyden–Fletcher–Goldfarb–Shanno algorithm (BFGS). The default method in the QuTiP control.pulseoptim GRAPE implemention is the L-BFGS-B method in Scipy, which is a wrapper to the implementation described in [4]. This limited memory and bounded method does not need to store the entire Hessian, which reduces the computer memory required, and allows bounds to be set for variable values, which considering these are field amplitudes is often physical.

It is typically far more efficient to calculate the gradients exactly rather than approximate them. For simple fidelity measures such as $f_{PSU}$ this is possible. Firstly the propagator gradient for each timeslot with respect to the control amplitudes is calculated. For closed systems, with unitary dynamics, a method using the eigendecomposition is used, which is efficient as it is this is also used in the propagator calculation (to exponentiate the Hamiltonian. More generally (for example open systems and symplectic dynamics) the Frechet derivative, or augmented matrix, method is used, which is described in [5]. For other optimisation goals it may not be possible to calculate analytic gradients. In these cases it is necessary to approximate the gradients, but this can be very slow, and can lead to other algorithms being faster.

QuTiP examples of GRAPE using second order gradient ascent methods are given in:  
- [pulseoptim Hadamard](http://nbviewer.ipython.org/github/qutip/qutip-notebooks/blob/master/examples/example-control-pulseoptim-Hadamard.ipynb)
- [pulseoptim QFT](http://nbviewer.ipython.org/github/qutip/qutip-notebooks/blob/master/examples/example-control-pulseoptim-QFT.ipynb)  
- Open systems: [pulseoptim - Lindbladian](http://nbviewer.ipython.org/github/qutip/qutip-notebooks/blob/master/examples/example-control-pulseoptim-Lindbladian.ipynb)  
- Symplectic dynamics: [pulseoptim - symplectic](http://nbviewer.ipython.org/github/qutip/qutip-notebooks/blob/master/examples/example-control-pulseoptim-symplectic.ipynb)

QuTiP examples of GRAPE using first order gradient ascent methods are given in:

- [GRAPE CNOT](http://nbviewer.ipython.org/github/qutip/qutip-notebooks/blob/master/examples/example-control-grape-cnot.ipynb)
- [GRAPE iSWAP](http://nbviewer.ipython.org/github/qutip/qutip-notebooks/blob/master/examples/example-control-grape-iswap.ipynb)
- [GRAPE Single-qubit rotation](http://nbviewer.ipython.org/github/qutip/qutip-notebooks/blob/master/examples/example-control-grape-single-qubit-rotation.ipynb)
- [GRAPE Toffoli gate](http://nbviewer.ipython.org/github/qutip/qutip-notebooks/blob/master/examples/example-control-grape-toffoli.ipynb)







# References
1. D. d’Alessandro, *Introduction to Quantum Control and Dynamics* (Chapman & Hall/CRC, 2008)
2. N. Khaneja, T. Reiss, C. Kehlet, T. Schulte-Herbruggen, and S. J. Glaser, J. Magn. Reson. 172, 296 (2005).
3. S. Machnes, U. Sander, S. J. Glaser, P. De Fouquieres, A. Gruslys, S. Schirmer, and T. Schulte-Herbrueggen, Phys. Rev. A 84, 1 (2010).
4. R. H. Byrd, P. Lu, J. Nocedal, and C. Zhu, SIAM J. Sci. Comput. 16, 1190 (1995).
5. F. F. Floether, P. de Fouquieres, and S. G. Schirmer, New J. Phys. 14, 073023 (2012).